## LR Scraper and Estimator

Overall, this notebook contains code to scrape diagnostic likelihood ratios from theNNT.com and convert them to numerical form. 

It also contains code to generate prompts for large language models to estimate the likelihood ratios. 

The output is a spreadsheet called: nnt_lrs_with_estimated which contains: 
- a sheet for each diagnosis or prediction target
- a row for each piece of information
- columns for the name, raw nnt lr, processed nnt lr, and estimated by 1 or more LLMs

### Extract Data from the NNT 

This scrapes all of the likelihood ratios listed on the NNT ('https://thennt.com/home-lr/') into Excel spreadsheets. 

1. A spreadsheet ("nnt_lrs.xlsx") contains a separate sheet for each page, which corresponds to a "prediction tasks" e.g. diagnosing the cause of a symptom - sometimes with specification of an intended population. Each sheet contains two columns: the name of the features (e.g. test result, finding, historical occurence, comorbditiy), the second contains the raw listing from the spreadsheet

2. A second spreadsheet contains the same sheets corresponding to a prediction target, and all of the features. These are used two call the 

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
import re
import math
import os
import json
import pandas as pd
from pydantic import BaseModel
from openai import OpenAI  # or your appropriate client wrapper
import math
from dotenv import load_dotenv

load_dotenv()  # looks for a .env file in the current dir by default
#print(os.getenv("OPENAI_API_KEY"))


True

In [ ]:
def get_specialty_links():
    """
    Extracts specialties and their corresponding article links from the webpage.
    Returns a list of dictionaries with specialty names and associated links.
    """

    url = 'https://thennt.com/home-lr/'
    response = requests.get(url)

    if response.status_code != 200:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'html.parser')

    # Locate the section with "Diagnosis (LR) Reviews by Specialty"
    specialty_section = soup.find('div', class_='well subdisplay accordion_caption', id='lr-byspecialty')

    if not specialty_section:
        print("Could not find the 'Diagnosis (LR) Reviews by Specialty' section on the webpage.")
        return []

    results = []

    # Find all specialty headings (e.g., h3)
    subheadings = specialty_section.find_all('h3')

    for subheading in subheadings:
        subheading_text = subheading.get_text(strip=True)  # Get specialty name
        links = []

        # Find the next unordered list (ul) which contains links
        next_ul = subheading.find_next_sibling('ul')

        if next_ul:
            for a_tag in next_ul.find_all('a', href=True):
                link_text = a_tag.get_text(strip=True)  # Link display name
                link_href = a_tag['href']  # Actual URL
                links.append({'display_name': link_text, 'url': link_href})

        results.append({'specialty': subheading_text, 'links': links})

    return results

def extract_likelihood_ratios(page_content):
    """
    Parses all likelihood ratio tables within <article class="lr_cards_details">.
    For each subsection indicated by an <h3> heading:
      - If the heading indicates Positive Findings, each finding will be prefixed with "Patient has: ".
      - If the heading indicates Negative Findings, a leading "No" (if present) is removed from the finding and then it is prefixed with "Patient does not have: ".
    This function processes all tables under a given heading (i.e. until the next <h3> is reached).
    Returns a list of tuples: (finding, likelihood ratio).
    """
    from bs4 import BeautifulSoup
    import re

    soup = BeautifulSoup(page_content, 'html.parser')
    results = []
    
    # Locate the main LR details section.
    lr_section = soup.find('article', class_='lr_cards_details')
    if not lr_section:
        return results

    # Find all <h3> headings in the section.
    headings = lr_section.find_all('h3')
    
    if headings:
        for h3 in headings:
            heading_text = h3.get_text(strip=True)
            if "Positive Findings" in heading_text:
                prefix = "Patient has: "
            elif "Negative Findings" in heading_text:
                prefix = "Patient does not have: "
            else:
                prefix = ""
            
            # Process all sibling elements until the next <h3> is encountered.
            sibling = h3.find_next_sibling()
            while sibling and sibling.name != "h3":
                if sibling.name == "table" and "lrtable" in sibling.get("class", []):
                    # Try to get proper data rows (i.e. <tr> elements with <td>).
                    rows = sibling.find_all("tr")
                    data_rows = [row for row in rows if row.find_all("td")]
                    
                    if data_rows:
                        for row in data_rows:
                            cols = row.find_all("td")
                            if len(cols) >= 2:
                                finding = cols[0].get_text(strip=True)
                                lr_value = cols[1].get_text(strip=True)
                                # If there's an <a> inside the LR cell, use its text.
                                link = cols[1].find("a")
                                if link:
                                    lr_value = link.get_text(strip=True) or lr_value
                                if not lr_value:
                                    lr_value = "Not reported"
                                
                                # Modify the finding string based on the prefix.
                                if prefix:
                                    if prefix.startswith("Patient does not have:"):
                                        finding = re.sub(r'^no\s+', '', finding, flags=re.IGNORECASE)
                                    finding = prefix + finding
                                
                                results.append((finding, lr_value))
                    else:
                        # If no rows with <td> are found, assume the table contains <td> elements in sequence.
                        all_tds = sibling.find_all("td")
                        # Process in pairs.
                        for i in range(0, len(all_tds), 2):
                            finding = all_tds[i].get_text(strip=True)
                            if i+1 < len(all_tds):
                                lr_value = all_tds[i+1].get_text(strip=True)
                            else:
                                lr_value = "Not reported"
                            # Check for an <a> element.
                            a_tag = all_tds[i+1].find("a")
                            if a_tag:
                                lr_value = a_tag.get_text(strip=True) or lr_value
                            if not lr_value:
                                lr_value = "Not reported"
                            
                            if prefix:
                                if prefix.startswith("Patient does not have:"):
                                    finding = re.sub(r'^no\s+', '', finding, flags=re.IGNORECASE)
                                finding = prefix + finding
                            results.append((finding, lr_value))
                sibling = sibling.find_next_sibling()
    else:
        # Fallback: process all tables in the section if no <h3> headings exist.
        tables = lr_section.find_all('table', class_='lrtable')
        for table in tables:
            rows = table.find_all("tr")
            data_rows = [row for row in rows if row.find_all("td")]
            if data_rows:
                for row in data_rows:
                    cols = row.find_all("td")
                    if len(cols) >= 2:
                        finding = cols[0].get_text(strip=True)
                        lr_value = cols[1].get_text(strip=True)
                        link = cols[1].find("a")
                        if link:
                            lr_value = link.get_text(strip=True) or lr_value
                        if not lr_value:
                            lr_value = "Not reported"
                        results.append((finding, lr_value))
            else:
                all_tds = table.find_all("td")
                for i in range(0, len(all_tds), 2):
                    finding = all_tds[i].get_text(strip=True)
                    if i+1 < len(all_tds):
                        lr_value = all_tds[i+1].get_text(strip=True)
                    else:
                        lr_value = "Not reported"
                    a_tag = all_tds[i+1].find("a") if i+1 < len(all_tds) else None
                    if a_tag:
                        lr_value = a_tag.get_text(strip=True) or lr_value
                    if not lr_value:
                        lr_value = "Not reported"
                    results.append((finding, lr_value))
                    
    return results

"""
def extract_likelihood_ratios(page_content):
    soup = BeautifulSoup(page_content, 'html.parser')
    results = []

    # Locate the section containing likelihood ratio tables
    lr_section = soup.find('article', class_='lr_cards_details')
    if not lr_section:
        return results  # Return empty if no section found

    # Find all tables inside the LR card
    tables = lr_section.find_all('table', class_='lrtable')

    for table in tables:
        # Grab all <tr> elements
        all_rows = table.find_all('tr')

        # Filter out any row that only has <th> (i.e., a header row)
        data_rows = []
        for row in all_rows:
            # If there's at least one <td> in this row, treat it as a data row
            if row.find_all('td'):
                data_rows.append(row)

        # If we have real data rows, parse them
        if data_rows:
            for row in data_rows:
                cols = row.find_all('td')
                # If the row has exactly 2 <td>, treat them as (finding, LR)
                if len(cols) == 2:
                    finding = cols[0].get_text(strip=True)
                    lr_value = cols[1].get_text(strip=True)
                    # If there's an <a> inside the LR cell, grab its text
                    link = cols[1].find('a')
                    if link:
                        lr_value = link.get_text(strip=True) or lr_value
                    if not lr_value:
                        lr_value = "Not reported"

                    results.append((finding, lr_value))

        else:
            # Fallback: if there are no valid data rows, we process all <td> in pairs
            cols = table.find_all('td')
            for i in range(0, len(cols) - 1, 2):
                finding = cols[i].get_text(strip=True)
                lr_value_element = cols[i + 1]

                # Extract the likelihood ratio, handling nested <a> and <br/>
                link = lr_value_element.find('a')
                if link:
                    lr_value = link.get_text(strip=True)
                else:
                    lr_value = lr_value_element.get_text(strip=True)

                if not lr_value:
                    lr_value = "Not reported"

                results.append((finding, lr_value))

    return results
"""
    
def fetch_webpages(specialty_links):
    """
    Iterates through all the extracted links, fetches the webpage content, 
    and extracts likelihood ratio findings.
    """
    findings_by_display_name = {}

    for item in specialty_links:
        print(f"Fetching pages for Specialty: {item['specialty']}")

        for link in item['links']:
            display_name = link['display_name']
            url = link['url']

            try:
                print(f"  - Fetching: {display_name} ({url})")
                response = requests.get(url)

                if response.status_code == 200:
                    print(f"    Success: {display_name} page fetched.")
                    
                    # Extract likelihood ratio findings
                    findings = extract_likelihood_ratios(response.text)
                    
                    # Store the extracted data
                    findings_by_display_name[display_name] = findings

                else:
                    print(f"    Failed to fetch {display_name} - Status Code: {response.status_code}")

                time.sleep(1)  # Optional: Add a delay to avoid overwhelming the server

            except requests.RequestException as e:
                print(f"    Error fetching {display_name}: {e}")

        print("\n")  # Add space between specialties for readability

    return findings_by_display_name

def save_to_excel(findings_data, filename="nnt_lrs.xlsx", blank_values=False):
    """
    Saves likelihood ratios to an Excel file with each display_name as a separate sheet.
    If blank_values is True, the Likelihood Ratio column is left blank.
    The first row contains the full display_name, and column headers start from the second row.
    """
    with pd.ExcelWriter(filename, engine="openpyxl") as writer:
        for display_name, findings in findings_data.items():
            if findings:
                # Prepare DataFrame
                df = pd.DataFrame(findings, columns=["Finding", "Likelihood Ratio"])

                if blank_values:
                    df["Likelihood Ratio"] = ""  # Clear likelihood ratio values

                # Insert full display_name as the first row
                full_name_row = pd.DataFrame({df.columns[0]: [display_name], df.columns[1]: [""]})
                df = pd.concat([full_name_row, df], ignore_index=True)

                # Save to Excel with sheet name as the **last** 31 characters
                sheet_name = display_name[-31:]
                df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)  # No default header

            else:
                print(f"Skipping {display_name} (No data found).")

    print(f"\nLikelihood ratios saved to {filename}")

# Fetch specialties and links
specialty_links = get_specialty_links()
findings_data = fetch_webpages(specialty_links)

# Save normal file
save_to_excel(findings_data, "nnt_lrs.xlsx", blank_values=False)

# Save version with blank likelihood ratios
save_to_excel(findings_data, "nnt_lrs_sans_number.xlsx", blank_values=True)

### Convert LR's to numerical format

This block takes in the excel spreadsheet with raw data from theNNT.com ("nnt_lrs.xlsx") and creates a new spreadsheet ("nnt_lrs_processed.xlsx") with a third column that contains a numerical version of the second column (raw data from theNNT) to be used as the LR_llm.


It removes any 'x's from the input, then determines whether the cell reports 

1. point estimate only (in which case use the point estimate)
2. point estimate + range (in which case take the point estimate), or 
3. range only (in which case, calculate the geometric mean)

It also counts the number of conditions (last 31 letters due to excel limitation) and LRs


[ ] TODO: 
- some of the BNP thresholds just have a number rather than a specification of the full "BNP > 100"; need to use header e.g. https://thennt.com/lr/dyspnea-due-to-heart-failure-without-chronic-respiratory-disease/ and https://thennt.com/lr/dyspnea-due-to-heart-failure-without-chronic-respiratory-disease/


In [ ]:
def parse_lr(lr_str):
    """
    Given a string from the 'Likelihood Ratio' cell, this function:
      - Removes any 'x' characters from the input.
      - If the string contains a parenthesized range (i.e. a point estimate plus a range),
        it returns the point estimate.
      - If the entire string is a range (e.g. "0.92-1.1", "3.3 to 4.8", "4.8–7.6"),
        it computes and returns the geometric mean.
      - Otherwise, it returns a float based on the first number found.
      - If the value is missing or cannot be parsed, returns NaN.
    """
    # Remove all 'x' characters and trim whitespace
    lr_str = lr_str.replace("x", "").strip()
    if lr_str == "":
        return np.nan

    # If parentheses exist, assume format "point_estimate (range)" and use the point estimate.
    if "(" in lr_str:
        point_part = lr_str.split("(")[0].strip()
        try:
            return float(point_part)
        except Exception:
            pass

    # Check for a range-only pattern.
    # This regex looks for two numbers separated by "to", "-" or "–" with optional whitespace.
    range_only_match = re.match(r'^\s*([0-9]*\.?[0-9]+)\s*(to|[-–])\s*([0-9]*\.?[0-9]+)\s*$', lr_str)
    if range_only_match:
        try:
            low = float(range_only_match.group(1))
            high = float(range_only_match.group(3))
            return math.sqrt(low * high)
        except Exception:
            return np.nan

    # Fallback: if no range-only pattern is found, extract the first number and return it.
    numbers = re.findall(r'([0-9]*\.?[0-9]+)', lr_str)
    if numbers:
        try:
            return float(numbers[0])
        except Exception:
            return np.nan

    return np.nan

# Load the original Excel file (each sheet has no header and the first row is the display name row)
input_filename = "nnt_lrs.xlsx"
output_filename = "nnt_lrs_processed.xlsx"

# Read all sheets from the Excel file into a dictionary of DataFrames.
excel_sheets = pd.read_excel(input_filename, sheet_name=None, header=None)

total_lr_count = 0
sheet_counts = {}

with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
    for sheet_name, df in excel_sheets.items():
        numerical_lr = []
        # Process each row in the sheet.
        for idx, row in df.iterrows():
            # For the header row (assumed to be the first row: condition label), add an empty string.
            if idx == 0:
                numerical_lr.append("")
            else:
                cell_value = row[1]  # The original "Likelihood Ratio" is in the second column (index 1)
                if pd.isna(cell_value) or str(cell_value).strip() == "":
                    numerical_lr.append("")
                else:
                    numerical_lr.append(parse_lr(str(cell_value)))
        
        # Insert the new column immediately after the "Likelihood Ratio" column.
        # This makes the new column the third column.
        df.insert(2, "Numerical LR", numerical_lr)
        
        # Remove rows (except the header) where the new "Numerical LR" is empty or NaN.
        header = df.iloc[[0]]  # Keep the header row (the condition label)
        data = df.iloc[1:]
        data = data[data["Numerical LR"].apply(lambda x: not (x == "" or pd.isna(x)))]
        filtered_df = pd.concat([header, data], ignore_index=True)
        
        # Insert a new row (after the condition label row) with the column labels.
        # The final sheet will have:
        #   Row 0: Condition label (from the original sheet)
        #   Row 1: Column labels: 'finding', 'lr_raw', and 'lr_num'
        #   Row 2+: Data rows
        col_labels = pd.DataFrame([["finding", "lr_raw", "lr_reported"]], columns=filtered_df.columns)
        final_df = pd.concat([filtered_df.iloc[[0]], col_labels, filtered_df.iloc[1:]], ignore_index=True)
        
        # Count the number of LR values for this sheet (exclude the two header rows).
        lr_count = len(final_df) - 2
        sheet_counts[sheet_name] = lr_count
        total_lr_count += lr_count
        
        # Write the modified DataFrame to the new Excel file.
        # The output maintains the original format: no index and no additional header row.
        final_df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

# Display counts.
num_sheets = len(excel_sheets)
print(f"Processed {num_sheets} condition(s) (sheets).")
for sheet, count in sheet_counts.items():
    print(f"Sheet '{sheet}' has {count} LR value(s).")
print(f"Total LR values processed across all sheets: {total_lr_count}.")

print(f"Processed Excel file saved as '{output_filename}'")

### Estimate LRs

NOTE: for the real run at this, we'll want to do some manual editing of the info columns - as there are some where it is a lab value that references the preceeding value (not currently automated to account for). 
--- particularly, BNP thresholds in the cardiac-cause sheets. 


This code block reads in the data from the nnt_lr_processed.xlsx excel file and calls a list of openAI models to have them give there best (single) estimate of the LR. Then, it rights a new spreadsheet nnt_lr_estimates that includes columns in each spreadsheet for each estimation. 

## Old Version

In [ ]:
# Define the response schema expecting a floating point number.
class LRResponse(BaseModel):
    value: float

def estimate_lr(diagnosis: str, info_val: str, client, model: str) -> float:
    """
    Calls the LLM with a prompt containing the diagnosis and a finding.
    Returns the estimated likelihood ratio as a floating point number.
    """
    lr_prompt = """You are an expert in medical diagnosis who is giving assessments of how important a piece of information is when determining whether a patient has a particularly condition. Your task is to estimate the likelihood ratio of a finding for a disease. Recall that the likelihood ratio represents how much the ratio between the odds of disease given a result for a lab value, whether a physical exam finding is present, or whether a comorbidity is present over the odds of disease when you did not know the result.
You will receive inputs in the following format; Target condition: <Condition, e.g. Patient Has: Cardiac chest pain>. Finding: <piece of information, e.g. ‘Patient does not have: radiation to the neck, arm, or jaw’>.
So, for example. If the odds of a Condition Z being present was 1 (meaning 50% probability) before we knew anything, but then we got a result (Finding A) it became 2 (meaning 2:1 odds or 66% probability), then the likelihood ratio would be 2. 
Given a condition and a finding, you will provide your best estimate of the likelihood ratio as a floating point number. Return your answer in valid JSON with the following schema: { 'value': <floating point number greater than 0> }.\n\n

Remember, stronger evidence in favor of a condition has a value farther above 1. Strong evidence against a diagnosis has a value farther below 1 (closer to 0). A likelihood ratio of 10 is equally strong evidence for a condition as a likelihood ratio of 0.1 is against it. Likelihood ratios near 1 represent weak evidence for or against. 
And if the "patient does not have: " some feature that is almost always present, that is strong evidence against.
(pay attention for double negatives- Patient has: no tobacco and Patient does not have: tobacco are identical)

Here is how I would like you to approach the problem:
First, consider the condition you are predicting (Condition: ___). Is the condition a medical diagnosis? If so, what kind of findings are usually present in someone who has that condition. Does the condition specify a certain type of patient? If so, how does that change things? 
Then, consider the finding. If a finding is much more common among patients who have the condition of interest than among patients who do not have the condition of interest, then the likelihood ratio should be high. This might be because the finding is a consequence of the disease, indicates that an enabling condition is present, indicates that a frequently comorbid condition is present, or is related to the pathology of the condition. In general, likelihood ratios over about 20 are pathognomonic, above 5 or so is extremely strong evidence in favor, above 2.5 or so is strong evidence, above 1.4 is so-so evidence, and 1-1.4 is pretty weak evidence. Conversely, if the finding is more common in people who do NOT have the condition, then the likelihood ratio should be below 1. Similarly, a likelihood ratio below 0.05 would exclude the condition in most situations, below 0.2 would be extremely strong evidence against, below 0.4 would be strong evidence against, below 0.71 is so-so, and between 0.71 and 1 is pretty weak evidence against (meaning, it just doesn’t change the odds of the condition much). 

Here are some hypothetical examples to consider: 
    Prompt = Target condition: Cardiac Chest Pain. Finding: Patient has: Pain not worse with exertion (requires they clarify exercise 1hr after meal).
    You would reason that because cardiac chest pain is usually worse with exertion because exertion worsens cardiac demand for oxygen, and thus worsens ischemia.
    Response = {
        ‘value’: 0.4
    }

    Prompt =  Target condition: Cardiac Chest Pain. Finding: Patient does not have: tobacco.
    You would reason that because being someone who smokes increases your risk of coronary artery disease, and thus being a never smoker means you’re at less risk… but many people who have heart attacks still smoke, so it’s only a weak predictor. 
    Response = {
        ‘value’: 0.75
    }

    Prompt = Target condition: Cardaic Chest Pain. Finding = Patient has: enjoys playing chess.
    You would reason that because enjoying chest has no relationship to having a heart attack.
    Response = {
        ‘value’: 1
    }

    Prompt = Target condition: Cardiac Chest Pain. Finding = Patient has: pain located behind the sternum
    You would reason that because cardiac chest pain is often experienced behind the sternum (thus, more likely), but so are many other causes of chest pain - like GERD.
    Response = {
        ‘value’: 1.2
    }

    Prompt = Condition: Cardiac Chest Pain. Finding = patient has: pain worse with exertion.
    You would reason that because the increased myocardial oxygen consumption worsens the pain if oxygen delivery to the myocardium is the cause, as it is in heart attacks.
    Response = {
        ‘value’: 3.4
    }

    OK: here’s the prompt.. """
    
    messages = [
        {"role": "system", "content": lr_prompt},
        {"role": "user", "content": f"Condition: {diagnosis}\nFinding: {info_val}"}
    ]

    # Check if the model starts with "o3-mini" "o3" "o4-mini, "o4", etc.
    kwargs = {}
    if model.startswith("o"):
        kwargs["reasoning_effort"] = "medium"  # low, medium, high depending on need

    # Call the LLM using the provided model name.
    completion = client.beta.chat.completions.parse(
        model=model,
        messages=messages,
        response_format=LRResponse,
        **kwargs  # pass the conditional keyword argument
    )

    # Call the LLM using the provided model name.
    completion = client.beta.chat.completions.parse(
        model=model,  # use the current model from the list
        messages=messages,
        response_format=LRResponse,
    )
    
    # Extract and return the floating point estimate.
    lr_response = completion.choices[0].message.parsed
    return lr_response.value


# Initialize the OpenAI (or your chosen) client using your API key.
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# List of model names to iterate over.
model_names = ['gpt-4o-mini-2024-07-18', 'gpt-4.1-mini-2025-04-14', 'gpt-4.1-2025-04-14', 'o4-mini-2025-04-16'] #'gpt-4o-2024-08-06', 'o3-mini-2025-01-31', gpt-4.1-2025-04-14, o4-mini-2025-04-16, 'o3-2025-04-16']

# Read the processed Excel file.
# We use header=None so that row 0 (the diagnosis row) and row 1 (the column headers row) are preserved.
input_filename = "nnt_lrs_processed.xlsx"
#input_filename = "new_" \
#"nnt_lrs_processed.xlsx" # use this one for the substitute in
sheets = pd.read_excel(input_filename, sheet_name=None, header=None)

# Process each sheet
for sheet_name, df in sheets.items():
    # Set diagnosis from the first row (row index 0, first cell)
    diagnosis = df.iloc[0, 0]

    # For each model in the list, call the LLM and add a new column with the estimation.
    for model in model_names:
        new_col_header = "lr_" + model
        new_col = []  # This list will hold one value per row
        print(f"Diagnosis: '{diagnosis}', Model: '{model}'")

        # Iterate over each row in the sheet.
        # Row 0 is the diagnosis row; row 1 is the existing column labels.
        for i in range(len(df)):
            if i == 0:
                new_col.append("")  # Leave the diagnosis row unchanged.
            elif i == 1:
                new_col.append(new_col_header)  # Insert the new column header in row 1.
            else:
                # For data rows, use the "finding" from the first column (index 0)
                info_val = df.iloc[i, 0]
                try:
                    estimated_lr = estimate_lr(diagnosis, info_val, client, model)
                except Exception as e:
                    estimated_lr = "ERROR"  
                    print(f"Error estimating LR for sheet '{sheet_name}', row {i}, model {model}: {e}")
                new_col.append(estimated_lr)
        
        # Insert the new column at the end of the dataframe.
        df.insert(df.shape[1], new_col_header, new_col)
    
    # Update the sheet data in our dictionary.
    sheets[sheet_name] = df

# Write out the modified sheets to a new Excel file.
output_filename = "nnt_lrs_with_estimated.xlsx"
with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
    for sheet_name, df in sheets.items():
        # Write without adding pandas default headers or indices.
        df.to_excel(writer, sheet_name=sheet_name, index=False, header=False)

print(f"Processed Excel file saved as '{output_filename}'")

## New Version

In [ ]:
"""
Bayesian LR estimator – tolerant of o‑family hidden‑reasoning bloat
2025‑07‑07
"""
import os, logging, pandas as pd
from pydantic import BaseModel
from openai import OpenAI

# --------------------------------------------------------------------------
# SYSTEM PROMPT – still enforces JSON-only output
# --------------------------------------------------------------------------
SYSTEM_PROMPT = (
    "You are a Bayesian-diagnostic assistant.\n"
    "Think step-by-step internally, then output only JSON:\n"
    '{"value": <float> (>0)}.'
)

# --------------------------------------------------------------------------
# INTERNAL (HIDDEN) REASONING CHECKLIST
# --------------------------------------------------------------------------
INTERNAL_CHECKLIST = (
    "Use this silent checklist:\n"
    " 1. Recall the typical presentation.\n"
    " 2. Consider if the findings is present or absent.\n"
    " 3. Compare frequency of the finding (or its absence) in positives vs negatives.\n"
    " 4. Use the LR guide as a rough reference.\n"
    " 5. Refine your estimate within the LR-guide categories (e.g. 0.3 vs 0.4).\n"
    " 6. Return JSON only."
)

LR_LADDER = (
    "LR guide ➜  >20 pathognomonic · 5-20 very strong for · "
    "2-5 moderate for · 1.4-2 weak for · 0.71-1.4 neutral · "
    "0.2-0.71 moderate against · 0.05-0.2 very strong against · <0.05 rule-out."
)

# ----------------  9-SHOT PRIMER  (for gpt-family)  -------------------------
RICH_PRIMER = """
<analysis># bucket: pathognomonic | step 1: prevalence high?
C: bacterial meningitis
F: nuchal rigidity
</analysis>
<json>{"value": 20}</json>

<analysis># bucket: very-strong-for
C: pulmonary embolism
F: Wells score >6
</analysis>
<json>{"value": 10}</json>

<analysis># bucket: strong-for
C: ectopic pregnancy
F: β-hCG >6500 IU + no intra-uterine sac
</analysis>
<json>{"value": 5}</json>

<analysis># bucket: weak-for
C: cardiac chest pain
F: pain behind sternum
</analysis>
<json>{"value": 1.5}</json>

<analysis># bucket: moderate-against
C: appendicitis
F: guarding absent
</analysis>
<json>{"value": 0.5}</json>

<analysis># bucket: very-strong-against
C: DKA
F: normal anion gap
</analysis>
<json>{"value": 0.1}</json>

<analysis># bucket: neutral
C: myocardial infarction
F: enjoys playing chess
</analysis>
<json>{"value": 1}</json>
""".strip()

# --------------  2-SHOT PRIMER  (for o-family)  ----------------------------
MINIMAL_PRIMER = """
<analysis># pathognomonic
C: bacterial meningitis
F: nuchal rigidity
</analysis>
<json>{"value": 20}</json>

<analysis># neutral
C: myocardial infarction
F: enjoys playing chess
</analysis>
<json>{"value": 1}</json>
""".strip()

# ------------ CAPABILITY MAP (single source‑of‑truth for all models) ---------

MODEL_CAPABILITIES = {
    # family  | length‑field              | temp? | hidden‑token caps to try
    "gpt-4o-mini-2024-07-18"  : {"field": "max_tokens",            "temp": True,  "caps": [64]},
    "gpt-4o-2024-08-06"       : {"field": "max_tokens",            "temp": True,  "caps": [64]},
    "gpt-4.1-mini-2025-04-14" : {"field": "max_tokens",            "temp": True,  "caps": [64]},
    "gpt-4.1-2025-04-14"      : {"field": "max_tokens",            "temp": True,  "caps": [64]},
    # o‑family – we escalate 512 → 1024 → 2048 if needed
    "o3-mini-2025-01-31": {"field": "max_completion_tokens", "temp": False,
                        "caps": [2048]},   # single generous cap
    "o3-2025-04-16":      {"field": "max_completion_tokens", "temp": False,
                        "caps": [2048]},
    "o4-mini-2025-04-16": {"field": "max_completion_tokens", "temp": False,
                        "caps": [2048]},
}
"""
# Abbreviated test run
MODEL_CAPABILITIES = {
    # family  | length‑field              | temp? | hidden‑token caps to try
    "gpt-4o-mini-2024-07-18"  : {"field": "max_tokens",            "temp": True,  "caps": [64]},
    "gpt-4.1-mini-2025-04-14" : {"field": "max_tokens",            "temp": True,  "caps": [64]}
}
"""


# ----------------------------- RESPONSE SCHEMA ------------------------------
class LRResponse(BaseModel):
    value: float

# --------------------------- HELPER FUNCTIONS -------------------------------
def build_messages(dx: str, finding: str, for_o: bool) -> list[dict]:
    primer = MINIMAL_PRIMER if for_o else RICH_PRIMER
    return [
        {"role": "system",    "content": SYSTEM_PROMPT},
        {"role": "system",    "content": INTERNAL_CHECKLIST},   # ← NEW
        {"role": "system",    "content": LR_LADDER},
        {"role": "assistant", "content": primer},
        {"role": "user",      "content": f"Condition: {dx}\nFinding: {finding}"},
    ]

def estimate_lr(diagnosis: str, finding: str, client: OpenAI, model: str) -> float:
    cfg          = MODEL_CAPABILITIES[model]
    is_reasoning = model.startswith("o")

    primer      = MINIMAL_PRIMER if is_reasoning else RICH_PRIMER
    temperature = 0.20 if is_reasoning else 0.10

    for cap in cfg["caps"]:
        try:
            completion = client.beta.chat.completions.parse(
                model=model,
                messages=build_messages(primer, diagnosis, finding),
                response_format=LRResponse,
                **{cfg["field"]: cap},
                **({"temperature": temperature} if cfg["temp"] else {}),
                **({"reasoning_effort": "medium"} if model.startswith("o3") else {}),
            )
            return float(completion.choices[0].message.parsed.value)
        except Exception as e:
            # Only retry on the length‑limit failure signature
            if "length limit was reached" in str(e) and cap != cfg["caps"][-1]:
                logging.warning(f"Retrying {model} with larger cap "
                                f"({cap}→{cfg['caps'][cfg['caps'].index(cap)+1]})")
                continue
            raise

# -------------------------------  MAIN PIPE  --------------------------------
logging.basicConfig(level=logging.WARNING)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

models = list(MODEL_CAPABILITIES)
input_file, output_file = "nnt_lrs_processed.xlsx", "nnt_lrs_with_estimated.xlsx"
sheets = pd.read_excel(input_file, sheet_name=None, header=None)

for sheet_name, df in sheets.items():
    diagnosis = df.iloc[0, 0]
    for model in models:
        new_header, col = "lr_" + model, []
        print(f"→ {diagnosis[:60]} | {model}")
        for i in range(len(df)):
            if i == 0:           col.append("")
            elif i == 1:         col.append(new_header)
            else:
                try:
                    lr = estimate_lr(diagnosis, df.iloc[i, 0], client, model)
                except Exception as e:
                    lr = "ERROR"
                    logging.warning(f"Error on sheet '{sheet_name}', row {i}, "
                                    f"model {model}: {e}")
                col.append(lr)
        df.insert(df.shape[1], new_header, col)
    sheets[sheet_name] = df

with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
    for name, frame in sheets.items():
        frame.to_excel(writer, sheet_name=name, index=False, header=False)

print(f"✅  All done – results saved to '{output_file}'")

## Newest Version (Aug 22nd)

GPT-5 Capabilities added and revised prompting strategy mildly.

In [ ]:
"""
Bayesian LR estimator — Responses API + GPT‑5–aware (no max_output_tokens)
Updated: 2025‑08‑22

Changes vs. your prior block:
• Responses API only (no Chat fallback).
• No max_output_tokens (output is schema‑constrained and tiny).
• Reasoning models use reasoning.effort="medium"; no temperature/top_p.
• GPT‑5(+mini) use text.verbosity="low" (applied only where supported).
• Includes these models: gpt‑5, gpt‑5‑mini, gpt‑4o‑mini‑2024‑07‑18, gpt‑4o‑2024‑08‑06,
  gpt‑4.1‑mini‑2025‑04‑14, gpt‑4.1‑2025‑04‑14, o3‑mini‑2025‑01‑31, o3‑2025‑04‑16, o4‑mini‑2025‑04‑16.
"""
from __future__ import annotations

import os
import logging
from pathlib import Path
from typing import Optional
import time, math
from random import random

import pandas as pd
from pydantic import BaseModel
from openai import OpenAI

# -----------------------------------------------------------------------------
# 0) Configuration
# -----------------------------------------------------------------------------
logging.basicConfig(level=logging.WARNING)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Model registry: whether it’s a reasoning model, whether it supports verbosity,
# and whether temperature is allowed.

MODEL_CAPABILITIES = {
    # GPT‑5 series (reasoning; supports text.verbosity; no temperature)
    "gpt-5"        : {"reasoning": True,  "verbosity": True,  "allow_temp": False},
    "gpt-5-mini"   : {"reasoning": True,  "verbosity": True,  "allow_temp": False},
    "gpt-5-nano"   : {"reasoning": True,  "verbosity": True,  "allow_temp": False},

    # GPT‑4.1 family (non‑reasoning; temperature OK); include snapshots + aliases
    "gpt-4.1-2025-04-14" : {"reasoning": False, "verbosity": False, "allow_temp": True},
    "gpt-4.1-mini-2025-04-14": {"reasoning": False, "verbosity": False, "allow_temp": True},
    "gpt-4.1-nano-2025-04-14": {"reasoning": False, "verbosity": False, "allow_temp": True},

    # GPT‑4o family (non‑reasoning; temperature OK); prefer latest snapshot or alias
    "gpt-4o-2024-11-20"  : {"reasoning": False, "verbosity": False, "allow_temp": True},
    "gpt-4o-mini-2024-07-18": {"reasoning": False, "verbosity": False, "allow_temp": True},

    # o‑series (reasoning; no temperature)
    "o3-2025-04-16" : {"reasoning": True,  "verbosity": False, "allow_temp": False},
    "o3-mini-2025-01-31": {"reasoning": True,  "verbosity": False, "allow_temp": False},
    "o4-mini-2025-04-16": {"reasoning": True,  "verbosity": False, "allow_temp": False},
}
MODELS = list(MODEL_CAPABILITIES)

# -----------------------------------------------------------------------------
INPUT_FILE  = "nnt_lrs_processed.xlsx"
OUTPUT_FILE = "nnt_lrs_with_estimated.xlsx"

# -----------------------------------------------------------------------------
# 1) Prompt (concise; no chain‑of‑thought)
# -----------------------------------------------------------------------------
SYSTEM_CORE = """You are a Bayesian diagnostic assistant.
Estimate a numeric likelihood ratio (LR) for a finding with respect to a diagnosis.
Return only a JSON object matching the schema: {"value": <float>}, where value > 0.
"""

DEFINITION = """Definition:
LR = P(finding | diagnosis) / P(finding | not-diagnosis)
"""

BANDS = """Reference bands (orientation):
>10 strong for; 5-10 moderate for; 2–5 weak for;
0.5–2 negligible;
0.2-0.5 weak against; 0.1-0.2 moderate against; ≤0.1 strong against"""

# Few‑shot examples (assistant outputs are JSON only)
FEW_SHOT_RICH = [
    ("subarachnoid hemorrhage", "xanthochromia present in CSF",                     19.0),
    ("pericarditis",            "pleuritic chest pain improved by leaning forward",  5.2),
    ("pulmonary embolism",      "tachycardia >100 bpm",                              2.2),
    ("urinary tract infection", "malodorous urine",                                  1.1),
    ("myocardial infarction",  "enjoys playing chess",                               1.0),
    ("appendicitis",            "no RLQ tenderness",                                0.45),
    ("pneumothorax",            "bilateral lung sliding present on US",             0.18),
    ("HIV infection",           "4th‑generation Ag/Ab screen negative beyond window",0.05),

]

FEW_SHOT_MIN = [
    ("subarachnoid hemorrhage", "xanthochromia present in CSF",                    19.0),
    ("myocardial infarction",  "enjoys playing chess",                             1.0),
]

def build_messages(diagnosis: str, finding: str, reasoning: bool) -> list[dict]:
    msgs: list[dict] = [
        {"role": "system", "content": SYSTEM_CORE.strip()},
        {"role": "system", "content": DEFINITION.strip()},
        {"role": "system", "content": BANDS.strip()},
    ]
    examples = FEW_SHOT_MIN if reasoning else FEW_SHOT_RICH
    for dx_ex, f_ex, v_ex in examples:
        msgs.append({"role": "user",      "content": f"Condition: {dx_ex}\nFinding: {f_ex}"})
        msgs.append({"role": "assistant", "content": f'{{"value": {float(v_ex)}}}'})
    msgs.append({"role": "user", "content": f"Condition: {diagnosis}\nFinding: {finding}"})
    return msgs

# -----------------------------------------------------------------------------
# 2) Structured Outputs schema (Pydantic)
# -----------------------------------------------------------------------------
class LRResponse(BaseModel):
    value: float

# -----------------------------------------------------------------------------
# 2b) Retry wrapper (exponential backoff with jitter)
# -----------------------------------------------------------------------------
def estimate_lr_until_positive(
    diagnosis: str,
    finding: str,
    model: str,
    client: Optional[OpenAI] = None,
    max_retries: Optional[int] = None,      # None ⇒ retry indefinitely
    base_backoff: float = 0.5,              # seconds
    max_backoff: float = 30.0               # seconds
) -> float:
    attempt = 0
    while True:
        attempt += 1
        try:
            lr = estimate_lr(diagnosis, finding, model, client)
            if isinstance(lr, (int, float)) and math.isfinite(lr) and lr > 0:
                return float(lr)
            raise ValueError(f"Non‑positive or non‑finite LR: {lr!r}")
        except Exception as e:
            logging.warning(
                f"[retry {attempt}] sheet finding='{finding[:80]}' | "
                f"model={model} → {e}"
            )
            if (max_retries is not None) and (attempt >= max_retries):
                raise
            # exponential backoff with jitter
            delay = min(base_backoff * (2 ** (attempt - 1)), max_backoff)
            time.sleep(delay * (0.5 + random()))  # 0.5–1.5× jitter

# -----------------------------------------------------------------------------
# 3) One‑call estimator (Responses API)
# -----------------------------------------------------------------------------
def estimate_lr(diagnosis: str, finding: str, model: str, client: Optional[OpenAI] = None) -> float:
    if client is None:
        client = OpenAI()

    cfg = MODEL_CAPABILITIES[model]
    msgs = build_messages(diagnosis, finding, reasoning=cfg["reasoning"])

    kwargs = {}
    if cfg["reasoning"]:
        kwargs["reasoning"] = {"effort": "medium"}     # for GPT‑5 and o‑series
        # no temperature/top_p
    elif cfg["allow_temp"]:
        kwargs["temperature"] = 0.2                    # allowed for 4o / 4.1

    # Apply verbosity only where supported (GPT‑5 family)
    if cfg["verbosity"]:
        kwargs["text"] = {"verbosity": "low"}

    resp = client.responses.parse(
        model=model,
        input=msgs,
        text_format=LRResponse,    # Structured Outputs → Pydantic
        **kwargs,
    )
    return float(resp.output_parsed.value)

# -----------------------------------------------------------------------------
# 4) Main pipeline: read workbook → append model columns → write output
# -----------------------------------------------------------------------------
def run_batch(input_file: str | Path, output_file: str | Path, models: list[str]) -> None:
    sheets = pd.read_excel(input_file, sheet_name=None, header=None)

    for sheet_name, df in sheets.items():
        diagnosis = str(df.iloc[0, 0]).strip()
        for model in models:
            new_header = "lr_" + model
            col = []
            print(f"→ {diagnosis[:60]} | {model}")
            for i in range(len(df)):
                if i == 0:
                    col.append("")                   # top-left cell (sheet label row)
                elif i == 1:
                    col.append(new_header)           # column header row
                else:
                    finding = str(df.iloc[i, 0]).strip()
                    if not finding:
                        col.append("")               # keep blank rows blank
                        continue
                    try:
                        # retry until a strictly positive, finite float is returned
                        lr = estimate_lr_until_positive(
                            diagnosis, finding, model, client,
                            max_retries=None         # set to an int (e.g., 8) to cap retries
                        )
                    except Exception as e:
                        lr = "ERROR"
                        logging.warning(
                            f"Error on sheet '{sheet_name}', row {i}, model {model} after retries: {e}"
                        )
                    col.append(lr)
            # Insert as object dtype to accommodate strings like "ERROR"
            df.insert(df.shape[1], new_header, pd.Series(col, dtype="object"))
        sheets[sheet_name] = df

    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        for name, frame in sheets.items():
            frame.to_excel(writer, sheet_name=name, index=False, header=False)

    print(f"Done – results saved to '{output_file}'")

if __name__ == "__main__":
    run_batch(INPUT_FILE, OUTPUT_FILE, MODELS)

→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | gpt-5
→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | gpt-5-mini
→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | gpt-5-nano
→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | gpt-4.1-2025-04-14
→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | gpt-4.1-mini-2025-04-14
→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | gpt-4.1-nano-2025-04-14
→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | gpt-4o-2024-11-20
→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | gpt-4o-mini-2024-07-18
→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | o3-2025-04-16
→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | o3-mini-2025-01-31
→ Diagnostic Accuracy of Ultrasound for Confirmation of Endotr | o4-mini-2025-04-16
→ Factors Predicting Difficult Endotracheal Intubation | gpt-5
→ Factors Predicting Difficult End

→ Factors Predicting Difficult Endotracheal Intubation | o3-mini-2025-01-31
→ Factors Predicting Difficult Endotracheal Intubation | o4-mini-2025-04-16
→ Acute Coronary Syndrome | gpt-5
→ Acute Coronary Syndrome | gpt-5-mini
→ Acute Coronary Syndrome | gpt-5-nano
→ Acute Coronary Syndrome | gpt-4.1-2025-04-14
→ Acute Coronary Syndrome | gpt-4.1-mini-2025-04-14
→ Acute Coronary Syndrome | gpt-4.1-nano-2025-04-14
→ Acute Coronary Syndrome | gpt-4o-2024-11-20
→ Acute Coronary Syndrome | gpt-4o-mini-2024-07-18
→ Acute Coronary Syndrome | o3-2025-04-16
→ Acute Coronary Syndrome | o3-mini-2025-01-31
→ Acute Coronary Syndrome | o4-mini-2025-04-16
→ Aortic Dissection | gpt-5
→ Aortic Dissection | gpt-5-mini
→ Aortic Dissection | gpt-5-nano
→ Aortic Dissection | gpt-4.1-2025-04-14
→ Aortic Dissection | gpt-4.1-mini-2025-04-14
→ Aortic Dissection | gpt-4.1-nano-2025-04-14
→ Aortic Dissection | gpt-4o-2024-11-20
→ Aortic Dissection | gpt-4o-mini-2024-07-18
→ Aortic Dissection | o3-2025-04-16
→ Ao

Code block to generate the processing notebook

In [4]:
import pandas as pd
from pathlib import Path

# ------------------------------------------------------------------ #
# 1.  Optional helper – keep the same “Feature Type” buckets   
#  [ ] TODO: this does not seem like it works very well - just usual manual for now #
# ------------------------------------------------------------------ #
def classify_feature_type(text: str) -> str:
    """Heuristic that matches the legacy categories."""
    t = str(text).lower()

    history = "history:" in t
    sign    = ("sign:" in t) or ("symptom" in t)
    score   = any(k in t for k in ("score", "points", "rule"))
    test    = any(k in t for k in ("test:", "lab", "troponin", "d‑dimer"))
    img     = any(k in t for k in (
        "ultrasound", "ct", "mri", "x‑ray", "radiograph", "imaging",
        "echo", "angiogram"))

    if history and test: return "History and Test"
    if history and img:  return "History and imaging"
    if history:          return "History_"
    if sign:             return "Sign_symptom"
    if score:            return "Score"
    if test:             return "Test finding"
    if img:              return "Imaging finding"
    return "Diagnosis"


# ------------------------------------------------------------------ #
# 2.  Converter – writes **one** sheet (same name as in template)     #
# ------------------------------------------------------------------ #
def convert_lr_workbook(
    *,                       # keyword‑only for clarity
    input_file: str | Path,  # e.g. "nnt_lrs_with_estimated.xlsx"
    template_file: str | Path,  # updated example workbook
    output_file: str | Path = "nnt_lrs_converted.xlsx",
) -> None:
    """
    • Collapses all per‑condition tabs from `input_file` into a master frame.
    • Adds the right‑most `condition` column (original tab name).
    • Writes a single worksheet whose name matches the (only) sheet
      found in `template_file`, with the header row stored as **row 1**
      (no Excel column headers) to preserve downstream‑notebook compatibility.
    """
    input_file    = Path(input_file)
    template_file = Path(template_file)
    output_file   = Path(output_file)

    # -------- determine the sole sheet name from the template --------------
    template_xls = pd.ExcelFile(template_file, engine="openpyxl")
    if len(template_xls.sheet_names) != 1:
        raise ValueError(
            f"Template has {len(template_xls.sheet_names)} sheets; "
            "expected exactly one after pruning."
        )
    target_sheet = template_xls.sheet_names[0]  # e.g. "Master"

    # ------------------- build master dataframe ----------------------------
    in_xls = pd.ExcelFile(input_file, engine="openpyxl")
    frames = []

    for tab in in_xls.sheet_names:
        raw = pd.read_excel(in_xls, sheet_name=tab, header=None)

        # Expect: row‑0 = diagnosis, row‑1 = column labels, row‑2+ = data
        if raw.shape[0] < 3:
            continue  # skip empty or malformed tabs

        header = raw.iloc[1]
        df = raw.iloc[2:].copy()
        df.columns = header
        df = df.loc[:, ~df.columns.isna()]  # drop unnamed columns

        # back‑fill Feature Type if the sheet didn't have it
        if "Feature Type" not in df.columns:
            df["Feature Type"] = df.iloc[:, 0].apply(classify_feature_type)

        df["condition"] = tab          # new right‑most column
        frames.append(df)

    if not frames:
        raise ValueError("No valid data found in the input workbook.")

    master = pd.concat(frames, ignore_index=True)
    # ensure 'condition' is the final column
    master = master[[c for c in master.columns if c != "condition"] + ["condition"]]

    # ------------------------ write single sheet ---------------------------
    with pd.ExcelWriter(output_file, engine="openpyxl") as writer:
        # replicate legacy layout: header row lives **inside** the block
        block = pd.concat(
            [pd.DataFrame([master.columns], columns=master.columns), master],
            ignore_index=True,
        )
        block.to_excel(
            writer,
            sheet_name=target_sheet[:31],  # Excel 31‑char cap
            index=False,
            header=False,
        )

    print(f"✅  Converted workbook written to: {output_file} "
          f"(single sheet: '{target_sheet}')")
    

# Run the conversion
convert_lr_workbook(
    input_file   = 'nnt_lrs_with_estimated.xlsx',   # produced after LR loop
    template_file= 'Past Runs/example_NNT_LRs_PC_07.03.2025.xlsx',  # updated example
    output_file  = 'new_NNT_LRs_08-23-2025.xlsx',
)

✅  Converted workbook written to: new_NNT_LRs_08-23-2025.xlsx (single sheet: 'Master')


Code block for trouble shooting the Scraper. 

In [ ]:
"""
# url = 'https://thennt.com/lr/dyspnea-due-to-heart-failure-without-chronic-respiratory-disease/'
url = 'https://thennt.com/lr/diagnostic-accuracy-history-physical-examination-laboratory-testing-giant-cell-arteritis/'
response = requests.get(url)

if response.status_code != 200:
    print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

soup = BeautifulSoup(response.text, 'html.parser')
print(soup)
"""